In [23]:
# -*- coding: utf-8 -*-
import google.generativeai as genai
import os

# Configure the API key
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

def load_email_content(file_path):
    """Load email content from a specified file."""
    try:
        with open(file_path, "r", encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return None
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return None

def create_prompt(prompt_type, content):
    """Create prompts based on the type and content provided."""
    prompts = {
        'extraction': f"""
Please extract the following components from the email content:

From: The sender's name and email address.
To: The recipient's name and email address.
CC: Any additional recipients (if applicable).
BCC: Any hidden recipients (if applicable).
Subject: The subject of the email.
Date: The date the email was sent.
Email Body: The main content of the email.

Here is the email content:
{content}

Output structure:
From: <Sender>
To: <Recipient>
CC: <CC Recipients>
BCC: <BCC Recipients>
Subject: <Email Subject>
Date: <Email Date>
Email Body: <Content>
""",
        'summary': f"Summarize the following email body:\n{content}",
        'translation': f"Translate the following summary to Hindi:\n{content}"
    }
    
    if prompt_type in prompts:
        return prompts[prompt_type]
    else:
        raise ValueError("Invalid prompt type specified.")

def generate_content(prompt):
    """Generate content using the generative model."""
    try:
        model = genai.GenerativeModel(model_name='gemini-1.5-flash')
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def main():
    # Load email content
    email_content = load_email_content("email.txt")
    
    if email_content:
        # Extract components
        extracted_info = generate_content(create_prompt('extraction', email_content))
        
        if extracted_info:
            print("Extracted Email Components:\n", extracted_info)
            email_body = extracted_info.split("Email Body:")[1].strip()  # Use only the email body
            
            # Generate summary from the email body
            summary = generate_content(create_prompt('summary', email_body))
            print("\nSummary:\n", summary)
            
            # Translate the summary to Hindi
            hindi_summary = generate_content(create_prompt('translation', summary))
            print("\nHindi Summary:\n", hindi_summary)

if __name__ == "__main__":
    main()


Extracted Email Components:
 From: Aniket wagh <aniket.wagh@example.com>
To: HR Department <hr@company.com>
CC: Team Lead <team.lead@company.com>
BCC: Yourself <aniket.wagh.bcc@example.com>
Subject: Application for Software Developer Position
Date: October 15, 2024
Email Body: 

Dear HR Team,

I hope this email finds you well. I am writing to express my interest in the Software Developer position at [Company Name]. With my background in full-stack development, proficiency in Python, React.js, Django, and experience in microservices architecture, I believe I would be a valuable asset to your team.

Please find my resume attached for your review. I would be grateful for the opportunity to further discuss how my skills align with your needs. I am available for an interview at your convenience.

Thank you for considering my application. I look forward to hearing from you.

Best regards,
Aniket wagh

Summary:
 Aniket Wagh is applying for the Software Developer position at [Company Name]. He

In [29]:
import json
import re
import os

# Define file paths for input and output
input_file_path = 'reviews_input.json'
output_file_path = 'reviews_output.json'

def read_input_reviews(file_path):
    """Read input reviews from a JSON file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error: The file '{file_path}' is not a valid JSON file.")
        return None

def write_output_reviews(file_path, data):
    """Write processed reviews to a JSON file."""
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, indent=2)

def process_reviews(reviews):
    """Process each review to extract sentiment and other details."""
    processed_reviews = []
    
    for entry in reviews:
        review_text = entry['review']
        item = entry.get('item', "unknown")
        brand = entry.get('brand', "unknown")
        sentiment, anger = analyze_review(review_text)
        
        review_dict = {
            "Original text": review_text,
            "Sentiment": sentiment,
            "Anger": anger,
            "Item": item,
            "Brand": brand
        }
        processed_reviews.append(review_dict)
    return processed_reviews

def analyze_review(review):
    """Analyze the review text for sentiment and anger."""
    positive_keywords = ['good', 'great', 'excellent', 'satisfied', 'love', 'fantastic', 'reliable', 'highly recommend', 'blown my mind', 'amazing']
    negative_keywords = ['bad', 'poor', 'terrible', 'angry', 'hate', 'disappointed', 'malfunctioning', 'cheap', 'waste', 'awful']
    
    sentiment = "unknown"
    anger = False

    # Check for sentiment
    review_lower = review.lower()
    if any(keyword in review_lower for keyword in positive_keywords):
        sentiment = "positive"
    elif any(keyword in review_lower for keyword in negative_keywords):
        sentiment = "negative"
        anger = "angry" in review_lower or "hate" in review_lower

    return sentiment, anger

def generate_email_response(review):
    """Generate an email response based on the review sentiment."""
    sentiment = review['Sentiment']
    original_text = review['Original text']
    if sentiment in ["positive", "neutral"]:
        return f"Dear Customer,\n\nThank you for your review: \"{original_text}\". We appreciate your feedback!\n\nBest regards,\nAI Customer Agent"
    elif sentiment == "negative":
        return f"Dear Customer,\n\nWe apologize for your experience with our product: \"{original_text}\". Please feel free to reach out to our customer service for assistance.\n\nBest regards,\nAI Customer Agent"
    else:
        return "Sentiment could not be determined."

def main():
    # Load input reviews from JSON file
    input_data = read_input_reviews(input_file_path)
    
    if input_data is None or "reviews" not in input_data:
        print("No valid reviews to process.")
        return
    
    reviews = input_data["reviews"]
    
    # Process reviews
    processed_reviews = process_reviews(reviews)
    
    # Prepare output with email responses
    output_data = []
    for review in processed_reviews:
        email_response = generate_email_response(review)
        review["Email Response"] = email_response
        output_data.append(review)
    
    # Write output to JSON file
    write_output_reviews(output_file_path, output_data)
    print(f"Processed reviews saved to '{output_file_path}'.")

if __name__ == "__main__":
    main()


Processed reviews saved to 'reviews_output.json'.
